In [1]:
import cv2
import os
import numpy as np
import re

In [2]:
class recognizer(object):

    def __init__(self):
        pass

    def recognize(self, model):
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier('./Cascade/haarcascade_frontalface_alt.xml')
        name = ["Akshay_real","Akshay_Photo"]
        
        #this is done to save result/record the video/output
        fourcc = cv2.VideoWriter_fourcc(*'MJPG') 
        out = cv2.VideoWriter('Face_Recognition.avi', fourcc, 20.0, (640, 480)) 


        while True:

            _, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y-20:y+h+20, x-20:x+w+20]

                try:
                    roi = cv2.resize(roi, (200,200), interpolation = cv2.INTER_LINEAR)
                    params = model.predict(roi)
#                     print(params)
                    cv2.putText(frame, name[params], (x, y-20), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 255, 0), 2)

                except:
                    continue
            cv2.imshow("Frame", frame)
            #this is done to save result/record the video/output
            out.write(frame)
            if cv2.waitKey(1) & 0xff == ord('q'):
                break

        cv2.destroyAllWindows()
        cap.release()

def read_database(path = "./database/"):

    images = os.listdir(path)
    img, idx = [], []
    for image in images:
        im = cv2.imread(path+image, cv2.IMREAD_GRAYSCALE)
        img.append(im)
#         idx.append(0)
#         print(re.search(r"([0-9]{1,3})", path+image).group(),' ',path+image)

        if int(re.search(r"([0-9]{1,3})", path+image).group())>149:
            idx.append(1)
        else:
            idx.append(0)

#         print(idx)

    return img, idx

if __name__ == "__main__":

    X,y = read_database()

    Obj = recognizer()
    
#     Eigenmodel = cv2.face.EigenFaceRecognizer_create()
#     Eigenmodel.train(np.asarray(X), np.asarray(y))
#     Obj.recognize(Eigenmodel)
    
#     Fishermodel = cv2.face.FisherFaceRecognizer_create()
#     Fishermodel.train(np.asarray(X), np.asarray(y))
#     Obj.recognize(Fishermodel)
    
    LBPHmodel = cv2.face.createLBPHFaceRecognizer()
    LBPHmodel.train(np.asarray(X), np.asarray(y))
    Obj.recognize(LBPHmodel)